<h1>MLTS Exercise 08 - Data Exploration</h1>

Your task is to read, explore and preprocess the following timeseries dataset. All information you will gather will be useful for the next notebook, were we will train a model based on this data.

The dataset can be downloaded from [Individual Household Electric Power Consumption](https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption)

It contains "Measurements of electric power consumption in one household with a one-minute sampling rate over a period of almost 4 years. Different electrical quantities and some sub-metering values are available."

**Reference**  
Hebrail, G. & Berard, A. (2006). Individual Household Electric Power Consumption [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C58K54.

In [1]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load the dataset

In [ ]:
# Importing dataset
path = 'data/household_power_consumption.txt'

Household_consumption = pd.read_csv(path, sep=';', low_memory=False)
Household_consumption

## Preprocess Data

* Convert seperate date and time columns into datetime column
* Convert numeric columns to correct type
* Find and replace missing values

In [3]:
# Parsing date and time into a single datetime column
Household_consumption['Datetime'] = pd.to_datetime(
    Household_consumption['Date'] + ' ' + Household_consumption['Time'], 
    format='%d/%m/%Y %H:%M:%S',
    errors='coerce'
)
# Drop date and time column
Household_consumption.drop(columns=['Date', 'Time'], inplace=True)
# Drop rows with missing datetime
Household_consumption.dropna(subset=['Datetime'], inplace=True)

In [ ]:
# Convert numerical columns to numeric type
numeric_columns = [
    'Global_active_power', 'Global_reactive_power', 'Voltage',
    'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'
]
for col in numeric_columns:
    Household_consumption[col] = pd.to_numeric(Household_consumption[col], errors='coerce')

Household_consumption.head(3)

In [ ]:
# Find missing values
missing_values = Household_consumption.isnull().sum()
print("Missing values per column:\n", missing_values)

# Fill missing values with median for simplicity
Household_consumption[numeric_columns] = Household_consumption[numeric_columns].fillna(
    Household_consumption[numeric_columns].median()
)

missing_values2 = Household_consumption.isnull().sum()
print("\nMissing values per column after getting rid of nul values:\n", missing_values2)


In [ ]:
# Summary statistics
print("\nSummary statistics:\n")
Household_consumption[numeric_columns].describe()

## Explore the data

Find different trends by looking at:
* Monthly Global Active Power
* Energy Usage Comparison Across Sub-Meterings
* Proportion of Total Energy Usage by Sub-Metering
* Total Global Active Power Consumption by Month
* Total Global Active Power Consumption by Day of the Week
* Total Global Active Power Consumption by Hour of the Day
* Average Hourly Global Active Power Consumption

Its recommended to use seaborn for some of these plots.

In [ ]:
# Extract month names for grouping
Household_consumption['Month'] = Household_consumption['Datetime'].dt.month_name()

# Order months correctly
month_order = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

# Plot boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(
    x='Month', y='Global_active_power', data=Household_consumption,
    order=month_order, palette="coolwarm", hue='Month', legend=False
)
plt.title("Monthly Trends of Global Active Power", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Global Active Power (kW)", fontsize=12)
plt.show()

In [ ]:
# Melt the sub-metering columns for a single boxplot comparison
sub_metering_data = Household_consumption.melt(
    id_vars=['Datetime'], 
    value_vars=['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'], 
    var_name='Sub-Metering', 
    value_name='Energy Consumption (Wh)'
)

# Plot boxplot
plt.figure(figsize=(8, 5))
sns.boxplot(
    x='Sub-Metering', y='Energy Consumption (Wh)', data=sub_metering_data,
    palette="Set2", hue='Sub-Metering', legend=False
)
plt.title("Energy Usage Comparison Across Sub-Meterings", fontsize=16)
plt.xlabel("Sub-Metering", fontsize=12)
plt.ylabel("Energy Consumption (Wh)", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.3)

# Show plot
plt.show()

In [ ]:
# Calculate total energy consumption for each sub-metering
sub_metering_totals = {
    'Sub-Metering 1 (Kitchen)': Household_consumption['Sub_metering_1'].sum(),
    'Sub-Metering 2 (Laundry)': Household_consumption['Sub_metering_2'].sum(),
    'Sub-Metering 3 (Water Heater/AC)': Household_consumption['Sub_metering_3'].sum(),
}

# Create the pie chart
plt.figure(figsize=(4, 4))
plt.pie(
    sub_metering_totals.values(),
    labels=sub_metering_totals.keys(),
    autopct='%1.1f%%',
    startangle=140,
    colors=sns.color_palette("pastel")
)

# Add title
plt.title("Proportion of Total Energy Usage by Sub-Metering", fontsize=16)

# Show plot
plt.show()

In [ ]:
# Group by Month and sum up the global_active_power
monthly_totals = Household_consumption.groupby('Month')['Global_active_power'].sum().reindex([
    "January", "February", "March", "April", "May", "June", 
    "July", "August", "September", "October", "November", "December"
])

# Plot the bar chart
plt.figure(figsize=(10, 4))
sns.barplot(x=monthly_totals.index, y=monthly_totals.values,palette="viridis",
            hue=monthly_totals.index, legend=False)
plt.title("Total Global Active Power Consumption by Month", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Energy Consumption (kW)", fontsize=12)
plt.xticks(rotation=45)

plt.show()

In [ ]:
# Add a Day of the Week column
Household_consumption['Day_of_Week'] = Household_consumption['Datetime'].dt.day_name()

# Group by Day of the Week and sum up the global_active_power
weekly_totals = Household_consumption.groupby('Day_of_Week')['Global_active_power'].sum().reindex([
    "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
])

# Plot the bar chart
plt.figure(figsize=(10, 5))
sns.barplot(x=weekly_totals.index, y=weekly_totals.values, palette="viridis",
            hue=weekly_totals.index, legend=False)
plt.title("Total Global Active Power Consumption by Day of the Week", fontsize=16)
plt.xlabel("Day of the Week", fontsize=12)
plt.ylabel("Total Energy Consumption (kW)", fontsize=12)
plt.show()

In [ ]:
# Add an Hour column
Household_consumption['Hour'] = Household_consumption['Datetime'].dt.hour

# Group by Hour and sum up the global_active_power
hourly_totals = Household_consumption.groupby('Hour')['Global_active_power'].sum()

# Plot the bar chart
plt.figure(figsize=(10, 4))
sns.barplot(x=hourly_totals.index, y=hourly_totals.values, palette="plasma",
            hue=hourly_totals.index, legend=False)
plt.title("Total Global Active Power Consumption by Hour of the Day", fontsize=16)
plt.xlabel("Hour of the Day", fontsize=12)
plt.ylabel("Total Energy Consumption (kW)", fontsize=12)
plt.xticks(range(0, 24), rotation=45)
plt.show()

In [ ]:
# Extract the hour of the day
Household_consumption['Hour'] = Household_consumption['Datetime'].dt.hour

# Group by hour and calculate average energy consumption
hourly_avg = Household_consumption.groupby('Hour')['Global_active_power'].mean()

# Plot hourly trends averaged over all days
plt.figure(figsize=(10, 4))
plt.plot(hourly_avg.index, hourly_avg.values, marker='o', linestyle='-', color='purple', linewidth=2)
plt.title("Average Hourly Global Active Power Consumption", fontsize=16)
plt.xlabel("Hour of the Day", fontsize=12)
plt.ylabel("Average Energy Consumption (kW)", fontsize=12)
plt.xticks(range(0, 24))
plt.grid(alpha=0.3)
plt.show()

## Time Series Specific Data Analysis

Look at the
* Daily Global Active Power Consumption with 7-Day Rolling Average
* Daily Global Active Power Consumption with 30-Day Rolling Average

In [ ]:
# Resample data to daily sums
daily_data_week = Household_consumption.resample('D', on='Datetime')['Global_active_power'].mean()

# Calculate a 7-day rolling average
daily_data_rolling = daily_data_week.rolling(window=7).mean()

# Plot daily trends and rolling average
plt.figure(figsize=(12, 6))
plt.plot(daily_data_week.index, daily_data_week, label="Daily Global Active Power", color='blue', alpha=0.6)
plt.plot(daily_data_rolling.index, daily_data_rolling, label="7-Day Rolling Average", color='red', linewidth=2)
plt.title("Daily Global Active Power Consumption with 7-Day Rolling Average", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Energy Consumption (kW)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# Resample the data to get daily sums
daily_data_month= Household_consumption.resample('D', on='Datetime')['Global_active_power'].mean()

# Calculate a 30-day rolling average
daily_data_rolling = daily_data_month.rolling(window=30).mean()

# Plot daily trends and 30-day rolling average
plt.figure(figsize=(12, 6))
plt.plot(daily_data_month.index, daily_data_month, label="Daily Global Active Power", color='blue', alpha=0.6)
plt.plot(daily_data_rolling.index, daily_data_rolling, label="30-Day Rolling Average", color='red', linewidth=2)
plt.title("Daily Global Active Power Consumption with 30-Day Rolling Average", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Energy Consumption (kW)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
